In [ ]:
import torch
import torch.nn.utils.prune as prune 
import numpy as np
import plotly.graph_objects as go 
import pandas as pd 
import torchvision



In [ ]:
crypto_name = 'apeusd'
#example_ohlc_df = pd.DataFrame({"Time": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],"Open": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],"High": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],"Low": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],"Close": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]})

ohlc_df = pd.read_csv('historical_price/' + crypto_name + '.csv')
ohlc_df.index = pd.to_datetime(ohlc_df['time'], unit = 'ms')
#ohlc_df.index = ohlc_df.index.tz_localize('UTC').tz_convert('US/Eastern')

In [ ]:
train_loader = torch.utils.data.DataLoader(torchvision.datasets.MNIST('../data', 
    train=True, 
    download=True,
    transform=torchvision.transforms.Compose),
    batch_size=1, 
    shuffle=True)

test_loader = torch.utils.data.DataLoader(, 
    train=False,
    transform=t
    batch_size=1,
     shuffle=True)


In [ ]:
class NeuralNetwork(torch.nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.fc1 = torch.nn.Linear(784, 1000) #input_size (784 becaues MNIST images are of dimension 28*28), hidden size
        self.fc2 = torch.nn.Linear(1000, 1000) #hidden size, output size
        self.fc3 = torch.nn.Linear(1000, 500) #hidden size, output size
        self.fc4 = torch.nn.Linear(500, 200) #hidden size, output size
        self.fc5 = torch.nn.Linear(200, 10) #hidden size, output size
     
    def forward(self,x):
        x = x.view(1, -1)
        x = self.fc1(x)
        x = torch.nn.functional.relu(x)
        x = self.fc2(x)
        x = torch.nn.functional.relu(x)
        x = self.fc3(x)
        x = torch.nn.functional.relu(x)
        x = self.fc4(x)
        x = torch.nn.functional.relu(x)
        x = self.fc5(x)
        #print(x.shape)
        return x
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
crypto_price_model = NeuralNetwork()
if torch.cuda.is_available():
    crypto_price_model.cuda()
crypto_price_model.to(device)

loss_function = torch.nn.CrossEntropyLoss() # Criterion, I believe CrossEntropyLoss has SoftMax built-in so I used it here, this is why my last layer doesn't have a softmax
optimizer  = torch.optim.Adam(crypto_price_model.parameters(), lr=0.001) # used adaptive moment estimation to optimize the model

In [ ]:
epochs = 2
min_validation_loss = np.inf
crypto_price_model.train() # set model to training mode
# if a GPU is available, use it
# if a model is in the directory, load it instead of training
"""Also referenced from https://www.geeksforgeeks.org/training-neural-networks-with-validation-using-pytorch/"""
for epoch in range(epochs):
    training_loss = 0.0    
    for data, labels in train_loader:
        if torch.cuda.is_available():
            data, labels = data.to(device), labels.to(device) # move to GPU, if available
        optimizer.zero_grad()
        target = crypto_price_model(data)
        loss = loss_function(target, labels)
        loss.backward()
        optimizer.step() # update the weights
        training_loss += loss.item()
    validation_loss = 0.0
    crypto_price_model.eval()
    for data, labels in test_loader:
        if torch.cuda.is_available():
            data, labels = data.to(device), labels.to(device) # move to GPU, if available
        target = crypto_price_model(data)
        loss = loss_function(target, labels)
        validation_loss += loss.item()*data.size(0)
    print(f'Epoch {epoch+1}/{epochs} Loss: {training_loss/len(train_loader)}')
    if min_validation_loss > validation_loss:
        print(f'Validation Loss Decreased({min_validation_loss:.6f}--->{validation_loss:.6f})\t Saving The Model')


In [ ]:
torch.save(crypto_price_model.state_dict(), f'{crypto_name}_model_weights.pth')

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
crypto_price_model.load_state_dict(torch.load(f'{crypto_name}_model_weights.pth', map_location=torch.device(device)))